<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第5课 数据收集 - Python网络爬虫实践 II </h1>

## 概述
接下来，我们通过一个更加复杂但是贴近实际的爬虫例子讲述。

这次爬取的内容是 IT桔子 中的新公司成立内容。也就是网站http://www.itjuzi.com/company?sortby=foundtime&page=1  的前10页。

爬取内容包括公司名、公司类别成立时间、省份、最新融资情况。最终以csv格式保存到文件。

> csv格式是一个常见的存储表格数据的格式，爬虫完成之后的csv文件，可以用excel直接打开。

## 注意
这个爬虫比之前的样例更加复杂，因为实际的网站中，可能在解析之后要通过一些字符串操作才能得到有效信息，如网页中常出现一些空格和换行来达到良好的显示效果，但是我们爬取的时候是要将这些字符去除。


In [4]:
import scrapy
import time
import csv
import os

class MySpider(scrapy.Spider):

    name = "spider"

    def __init__(self):
        self.file = open('demo2_newCompanies.csv', 'w',
                         encoding='GBK', newline='')
        self.csvWriter = csv.DictWriter(
            self.file, fieldnames=['name', 'type', 'date', 'province'])

        # 设置待爬取网站列表
        self.urls = []
        for i in range(1, 10):
            self.urls.append(
                'http://www.itjuzi.com/company?sortby=foundtime&page=' + str(i))
        print(self.urls)

    def start_requests(self):
        # self.init_urls()
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)

    # parse方法会在每个request收到response之后调用
    def parse(self, response):

        # print(response.body)

        # 提取公司列表
        companys = response.css(".list-main-icnset.company-list-ul li")

        # 从一开始是为了跳过网页内的表格标题栏
        for company in companys:
            # 解析公司名
            name = company.css(".title span::text").extract_first()
            # 跳过没有公司名的公司
            if name is None:
                continue

            # 解析公司大类
            type = company.css(".cell.classify::text").extract_first()
            # 去除网页原有的空格、换行、制表符
            type = type.replace('\t', '').replace('\n', '').replace(' ', '')
            # 解析得到时间
            date = company.css(".date::text").extract_first()
            # 去除网页原有的空格、换行、制表符
            date = date.replace('\t', '').replace('\n', '').replace(' ', '')
            # 解析省份
            province = company.css(".cell.place::text").extract_first()
            province = province.replace('\t', '').replace(
                '\n', '').replace(' ', '')
            # 构建字典
            item = {"name": name, "type": type, "date": date,
                    "province": province}

            # 以csv格式写入文件
            self.csvWriter.writerow(item)

        # 及时将内容写入文件，否则可能会出现少许延迟
        self.file.flush()
        os.fsync(self.file)
        # 输出当前解析完成的网页网址，可以当做爬取进度来看待,与程序逻辑无关
        print("over: " + response.url)

In [5]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
process.start() # 这句代码就是开始了整个爬虫过程 

2017-11-14 20:58:56 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2017-11-14 20:58:56 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2017-11-14 20:58:56 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-14 20:58:56 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-14 20:58:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2017-11-14 20:58:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2017-11-14 20:58:56 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtime

['http://www.itjuzi.com/company?sortby=foundtime&page=1', 'http://www.itjuzi.com/company?sortby=foundtime&page=2', 'http://www.itjuzi.com/company?sortby=foundtime&page=3', 'http://www.itjuzi.com/company?sortby=foundtime&page=4', 'http://www.itjuzi.com/company?sortby=foundtime&page=5', 'http://www.itjuzi.com/company?sortby=foundtime&page=6', 'http://www.itjuzi.com/company?sortby=foundtime&page=7', 'http://www.itjuzi.com/company?sortby=foundtime&page=8', 'http://www.itjuzi.com/company?sortby=foundtime&page=9']


ReactorNotRestartable: 